In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
!mkdir ~/.kaggle
!mv kaggle.json ~/.kaggle/


In [ ]:
!kaggle competitions download -c playground-series-s4e2

In [7]:
import zipfile

# Ruta al archivo comprimido
zip_file_path = "/content/playground-series-s4e2.zip"

# Directorio de destino para la extracción
extract_dir = "/content/sample_data"

# Descomprimir el archivo
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)


In [8]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Load the data
train_data = pd.read_csv("/content/sample_data/train.csv")
test_data = pd.read_csv("/content/sample_data/test.csv")

# Separate features and target variable
X_train = train_data.drop(columns=["id", "NObeyesdad"])
y_train = train_data["NObeyesdad"]
X_test = test_data.drop(columns=["id"])

# One-hot encode categorical features for both training and test data
X_train_encoded = pd.get_dummies(X_train)
X_test_encoded = pd.get_dummies(X_test)

# Align the columns in the test dataset with those in the training dataset
missing_cols = set(X_train_encoded.columns) - set(X_test_encoded.columns)
for col in missing_cols:
    X_test_encoded[col] = 0

# Ensure the order of columns is the same
X_test_encoded = X_test_encoded[X_train_encoded.columns]

# Split the training data into training and validation sets
X_train_final, X_valid, y_train_final, y_valid = train_test_split(X_train_encoded, y_train, test_size=0.2, random_state=42)

# Define the classifier
rf = RandomForestClassifier(random_state=42)

# Define the hyperparameters to tune
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10]
}

# Perform grid search cross-validation
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train_final, y_train_final)

# Get the best model
best_rf = grid_search.best_estimator_

# Evaluate the best model
best_rf.fit(X_train_final, y_train_final)
best_rf_pred = best_rf.predict(X_valid)
best_accuracy = accuracy_score(y_valid, best_rf_pred)
print("Best Model Accuracy:", best_accuracy)

# Make predictions on test data using the best model
test_predictions = best_rf.predict(X_test_encoded)

# Create submission DataFrame
submission = pd.DataFrame({"id": test_data["id"], "NObeyesdad": test_predictions})

# Save submission file
submission.to_csv("/content/sample_data/submission.csv", index=False)

Best Model Accuracy: 0.8908959537572254
